In [36]:
# Check GPU version
!nvidia-smi

Thu Jun  3 21:51:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.31       Driver Version: 462.31       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX250      WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   42C    P8    N/A /  N/A |     64MiB /  2048MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [37]:
# Check CUDA/cuDNN Version
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:04_Central_Daylight_Time_2018
Cuda compilation tools, release 10.0, V10.0.130


'which' 不是內部或外部命令、可執行的程式或批次檔。


In [ ]:
# install packages
!pip install catboost
!pip install xgboost
!pip install pycaret[full]
!pip install wandb

In [ ]:
# 即時監控colab資源
import wandb

# connect with jim107225017/colab/20210526
wandb.init(project='colab', entity='jim107225017', name='CPU_GPU', id='20210526')

In [ ]:
# install lightgbm GPU in colab
# 先登入google cloud
from google.colab import drive
drive.mount('/content/drive')

!pip uninstall lightgbm -y
!git clone --recursive https://github.com/Microsoft/LightGBM
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu

In [2]:
# import packages
import os
import numpy as np
import pandas as pd

# Pycaret
from pycaret.classification import *

import catboost
import lightgbm
import xgboost

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from joblib import load, dump

In [3]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/colab/TPS May"
path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - Apr 2021'
os.chdir(path)

In [4]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
df_train.isnull().sum()

PassengerId        0
Survived           0
Pclass             0
Name               0
Sex                0
Age             3292
SibSp              0
Parch              0
Ticket          4623
Fare             134
Cabin          67866
Embarked         250
dtype: int64

In [6]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


In [7]:
all_data = pd.concat([df_train, df_test]).reset_index(drop=True)

In [8]:
# 1.Missing value of Age = mean
vari = 'Age'
print(f"Type {vari} is {all_data[vari].dtype}!!!")

print(all_data[vari].describe())

print(f"Missing ratio for train is {df_train[vari].isnull().sum() / len(df_train):.3f}")
print(f"Missing ratio for test is {df_test[vari].isnull().sum() / len(df_test):.3f}")

all_data[vari] = all_data[vari].fillna(all_data[vari].mean())

print(all_data[vari].isnull().sum())

Type Age is float64!!!
count    193221.000000
mean         34.464565
std          16.783847
min           0.080000
25%          22.000000
50%          31.000000
75%          48.000000
max          87.000000
Name: Age, dtype: float64
Missing ratio for train is 0.033
Missing ratio for test is 0.035
0


In [9]:
# 2.Missing value of Ticket = 'X', split string and take first part
vari = 'Ticket'
print(f"Type {vari} is {all_data[vari].dtype}!!!")

print(all_data[vari].describe())

print(f"Missing ratio for train is {df_train[vari].isnull().sum() / len(df_train):.3f}")
print(f"Missing ratio for test is {df_test[vari].isnull().sum() / len(df_test):.3f}")

all_data[vari] = all_data[vari].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

print(f'There is {all_data[vari].nunique()} unique values in array : {all_data[vari].unique()} ')

print(all_data[vari].isnull().sum())

Type Ticket is object!!!
count     190196
unique    132613
top         A/5 
freq         646
Name: Ticket, dtype: object
Missing ratio for train is 0.046
Missing ratio for test is 0.052
There is 50 unique values in array : ['X' 'CA' 'A.' 'A/S' 'PC' 'W./C.' 'SC/PARIS' 'S.C./PARIS' 'SC/Paris' 'CA.'
 'SOTON/O.Q.' 'C.A.' 'A/5.' 'STON/O' 'A/4' 'C' 'AQ/4' 'STON/O2.' 'WE/P'
 'F.C.' 'F.C.C.' 'PP' 'S.O.C.' 'SC/AH' 'Fa' 'W.E.P.' 'C.A./SOTON' 'P/PP'
 'A/5' 'SOTON/O2' 'SW/PP' 'STON/OQ.' 'W/C' 'S.O./P.P.' 'SC' 'A./5.' 'A/4.'
 'S.O.P.' 'SOTON/OQ' 'SO/C' 'SCO/W' 'A.5.' 'S.W./PP' 'S.P.' 'LP' 'SC/A4'
 'AQ/3.' 'S.C./A.4.' 'A4.' 'SC/A.3'] 
0


In [10]:
# 3.Missing value of Fare = median by each Pclass and use log transform
vari = 'Fare'
print(f"Type {vari} is {all_data[vari].dtype}!!!")

print(all_data[vari].describe())

print(f"Missing ratio for train is {df_train[vari].isnull().sum() / len(df_train):.3f}")
print(f"Missing ratio for test is {df_test[vari].isnull().sum() / len(df_test):.3f}")

fare_map = all_data[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
all_data['Fare'] = all_data['Fare'].fillna(all_data['Pclass'].map(fare_map['Fare']))
all_data['Fare'] = np.log1p(all_data['Fare'])   # log1p = log(x+1) : Gaussian transform


print(all_data[vari].isnull().sum())

Type Fare is float64!!!
count    199733.000000
mean         44.652071
std          67.436104
min           0.050000
25%          10.080000
50%          20.250000
75%          34.850000
max         744.660000
Name: Fare, dtype: float64
Missing ratio for train is 0.001
Missing ratio for test is 0.001
0


In [11]:
# 4.Missing value of Cabin = 'X' and take first letter
vari = 'Cabin'
print(f"Type {vari} is {all_data[vari].dtype}!!!")

print(all_data[vari].describe())

print(f"Missing ratio for train is {df_train[vari].isnull().sum() / len(df_train):.3f}")
print(f"Missing ratio for test is {df_test[vari].isnull().sum() / len(df_test):.3f}")

all_data[vari] = all_data[vari].fillna('X').map(lambda x: x[0].strip())

print(f'There is {all_data[vari].nunique()} unique values in array : {all_data[vari].unique()} ')

print(all_data[vari].isnull().sum())

# or drop the feature directly
# for d in [df_train, df_test]:
#     d.drop(vari, axis=1, inplace=True)

Type Cabin is object!!!
count      61303
unique     45442
top       C11139
freq           7
Name: Cabin, dtype: object
Missing ratio for train is 0.679
Missing ratio for test is 0.708
There is 9 unique values in array : ['C' 'X' 'A' 'D' 'B' 'E' 'F' 'G' 'T'] 
0


In [12]:
# 5.Missing value of Embarked = 'X'
vari = 'Embarked'
print(f"Type {vari} is {all_data[vari].dtype}!!!")

print(all_data[vari].describe())

print(f"Missing ratio for train is {df_train[vari].isnull().sum() / len(df_train):.3f}")
print(f"Missing ratio for test is {df_test[vari].isnull().sum() / len(df_test):.3f}")

all_data[vari] = all_data[vari].fillna('X')

print(f'There is {all_data[vari].nunique()} unique values in array : {all_data[vari].unique()} ')

print(all_data[vari].isnull().sum())

Type Embarked is object!!!
count     199473
unique         3
top            S
freq      140981
Name: Embarked, dtype: object
Missing ratio for train is 0.003
Missing ratio for test is 0.003
There is 4 unique values in array : ['S' 'C' 'Q' 'X'] 
0


In [13]:
# 6.Name, take only surnames
vari = 'Name'
all_data[vari] = all_data[vari].map(lambda x: x.split(',')[0])

print(f'There is {all_data[vari].nunique()} unique values in array : {all_data[vari].unique()} ')

There is 26470 unique values in array : ['Oconnor' 'Bryan' 'Owens' ... 'Pecatoste' 'Conlisk' 'Peitz'] 


In [14]:
# Encoding and transform
label_cols = ['Name', 'Ticket', 'Sex']
onehot_cols = ['Cabin', 'Embarked']
num_cols = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
TARGET = 'Survived'

def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

scaler = StandardScaler()
minmax = MinMaxScaler()

cat_df = all_data[onehot_cols]

lab_df = all_data[label_cols].apply(label_encoder)
lab_df = pd.DataFrame(minmax.fit_transform(lab_df[label_cols]), columns=label_cols)

num_df = pd.DataFrame(scaler.fit_transform(all_data[num_cols]), columns=num_cols)

target_df = all_data[TARGET]

all_data = pd.concat([num_df, lab_df, cat_df, target_df], axis=1)

# split train and test
df_train = all_data[:len(df_train)]
df_test = all_data[len(df_train):].drop(TARGET, axis=1)

In [15]:
print(df_train.head())
print(df_train.shape)

     Pclass           Age      Fare     SibSp     Parch      Name    Ticket  \
0 -1.425730 -8.614253e-16  0.134351  1.901268 -0.505478  0.658922  1.000000   
1  0.877699 -8.614253e-16 -0.533837 -0.539572 -0.505478  0.115720  1.000000   
2  0.877699 -2.069149e+00  1.070483  0.680848  1.628715  0.672409  0.285714   
3  0.877699 -9.374220e-01 -0.555506 -0.539572 -0.505478  0.481393  0.000000   
4  0.877699 -5.737175e-01 -1.023540 -0.539572 -0.505478  0.088216  1.000000   

   Sex Cabin Embarked  Survived  
0  1.0     C        S       1.0  
1  1.0     X        S       0.0  
2  1.0     X        S       0.0  
3  1.0     X        S       0.0  
4  1.0     X        S       1.0  
(100000, 11)


In [16]:
# Set-Up
PROJECT_NAME = 'pycaret-20210603'
SEED = 20210603

setup(data = df_train[label_cols + onehot_cols + num_cols + [TARGET]],
      categorical_features=onehot_cols,
      numeric_features=label_cols + num_cols,
      imputation_type='iterative',
      target = TARGET,
      train_size=0.8,
      data_split_shuffle=True,
      data_split_stratify=True,
      fold = 5,
      fold_strategy='stratifiedkfold',   # or 'kfold', 'stratifiedkfold', 'groupkfold', 'timeseries'
      fold_shuffle = True,
      n_jobs=-1,
      use_gpu = True,
      session_id=SEED,   # seed
      silent = True)

,Description,Value
0,session_id,20210603
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(100000, 11)"
5,Missing Values,False
6,Numeric Features,8
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


({'acc': <pycaret.containers.metrics.classification.AccuracyMetricContainer at 0x240e1d0c588>,
  'auc': <pycaret.containers.metrics.classification.ROCAUCMetricContainer at 0x240e50ba848>,
  'recall': <pycaret.containers.metrics.classification.RecallMetricContainer at 0x240e3b1a248>,
  'precision': <pycaret.containers.metrics.classification.PrecisionMetricContainer at 0x240e50bac88>,
  'f1': <pycaret.containers.metrics.classification.F1MetricContainer at 0x240e50bad88>,
  'kappa': <pycaret.containers.metrics.classification.KappaMetricContainer at 0x240e50baf08>,
  'mcc': <pycaret.containers.metrics.classification.MCCMetricContainer at 0x240e50baf48>},
 1,
 False,
 'clf-default-name',
 Pipeline(memory=None, steps=[('empty_step', 'passthrough')], verbose=False),
 [],
 <MLUsecase.CLASSIFICATION: 1>,
 True,
 StratifiedKFold(n_splits=5, random_state=20210603, shuffle=True),
 'box-cox',
 Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_featu

In [17]:
# GPU Enable
models(internal=True)[['Name', 'GPU Enabled']]

,Name,GPU Enabled
ID,,
lr,Logistic Regression,False
knn,K Neighbors Classifier,False
nb,Naive Bayes,False
dt,Decision Tree Classifier,False
svm,SVM - Linear Kernel,False
rbfsvm,SVM - Radial Kernel,False
gpc,Gaussian Process Classifier,False
mlp,MLP Classifier,False
ridge,Ridge Classifier,False


In [36]:
top6 = compare_models(exclude=['catboost'],   #catboost不能 tune_model
                      sort='Accuracy',   # or ['Accuracy', 'AUC', 'Recall', 'Precision', 'F1', 'Kappa', 'MCC']
                      n_select=6,
                      cross_validation=True,
                      budget_time=10*60,   # in minutes
                      )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7828,0.8516,0.7491,0.7447,0.7469,0.5567,0.5568,6.6640
lightgbm,Light Gradient Boosting Machine,0.7826,0.8534,0.7420,0.7478,0.7449,0.5555,0.5555,2.4960
xgboost,Extreme Gradient Boosting,0.7777,0.8484,0.7299,0.7453,0.7375,0.5448,0.5449,2.2980
ada,Ada Boost Classifier,0.7769,0.8452,0.7040,0.7574,0.7297,0.5402,0.5413,1.8220
lr,Logistic Regression,0.7751,0.8371,0.7436,0.7340,0.7388,0.5413,0.5414,0.5520
ridge,Ridge Classifier,0.7747,0.0000,0.7595,0.7264,0.7426,0.5425,0.5430,0.0720
lda,Linear Discriminant Analysis,0.7746,0.8368,0.7598,0.7261,0.7425,0.5423,0.5428,0.2180
rf,Random Forest Classifier,0.7670,0.8330,0.7124,0.7349,0.7234,0.5223,0.5225,5.4400
et,Extra Trees Classifier,0.7603,0.8259,0.7036,0.7272,0.7152,0.5083,0.5086,9.3460
svm,SVM - Linear Kernel,0.7584,0.0000,0.7310,0.7119,0.7213,0.5082,0.5083,0.2880


In [37]:
# Tune Top 6 models
tuned_top6 = [tune_model(i, n_iter=20, search_library="optuna", search_algorithm="tpe") for i in top6]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7725,0.0000,0.7528,0.7256,0.7390,0.5375,0.5378
1,0.7757,0.0000,0.7694,0.7236,0.7458,0.5454,0.5463
2,0.7750,0.0000,0.7563,0.7282,0.7420,0.5426,0.5429
3,0.7765,0.0000,0.7572,0.7303,0.7435,0.5456,0.5459
4,0.7746,0.0000,0.7622,0.7249,0.7431,0.5426,0.5431
Mean,0.7749,0.0000,0.7596,0.7265,0.7427,0.5427,0.5432
SD,0.0013,0.0000,0.0058,0.0024,0.0022,0.0029,0.0030


In [45]:
# stacking
stacker = stack_models(tuned_top6)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7782,0.8508,0.7250,0.7486,0.7366,0.5452,0.5454
1,0.7834,0.8548,0.7405,0.7499,0.7452,0.5568,0.5568
2,0.7845,0.8535,0.7329,0.7559,0.7442,0.5581,0.5583
3,0.7861,0.8567,0.7298,0.7605,0.7449,0.5609,0.5612
4,0.7811,0.8532,0.7355,0.7484,0.7419,0.5519,0.5520
Mean,0.7827,0.8538,0.7328,0.7527,0.7426,0.5546,0.5548
SD,0.0027,0.0019,0.0052,0.0048,0.0032,0.0055,0.0055


In [46]:
# blending
blender = blend_models(tuned_top6)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7780,0.0000,0.7268,0.7473,0.7369,0.5450,0.5451
1,0.7804,0.0000,0.7408,0.7445,0.7426,0.5511,0.5511
2,0.7831,0.0000,0.7347,0.7525,0.7435,0.5557,0.5558
3,0.7812,0.0000,0.7269,0.7530,0.7397,0.5511,0.5513
4,0.7788,0.0000,0.7275,0.7482,0.7377,0.5465,0.5466
Mean,0.7803,0.0000,0.7313,0.7491,0.7401,0.5499,0.5500
SD,0.0018,0.0000,0.0056,0.0032,0.0026,0.0038,0.0038


In [47]:
# select best model 
best = automl(optimize = 'Accuracy')
print(best)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=20210603, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [48]:
# save best, stacking, blending and top6 model
save_model(best, model_name=PROJECT_NAME)
save_model(stacker, model_name=PROJECT_NAME + '_stacking')
save_model(blender, model_name=PROJECT_NAME + '_blending')
save_model(tuned_top6, model_name=PROJECT_NAME + '_top6')

Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['Cabin',
                                                             'Embarked'],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['Name', 'Ticket',
                                                           'Sex', 'Pclass',
                                                           'Age', 'Fare',
                                                           'SibSp', 'Parch'],
                                       target='Survived', time_features=[])),
                 ('imputer',
                  Iterative_Imputer(add_indicator=False,
                                    c...
                                      intercept_scaling=1, l1_ratio=None,
                   

In [49]:
# load best model
best = load_model(PROJECT_NAME)

Transformation Pipeline and Model Successfully Loaded


In [50]:
# predict
result = predict_model(best, data=df_test, raw_score=False)   # predict
result['Label'] = result['Label'].astype('float64').astype('int64')

In [101]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result['Label'].values.reshape(-1,1)
sub.to_csv(f'{PROJECT_NAME}.csv', index=False)

In [102]:
# load stacking model
stacker = load_model(PROJECT_NAME + '_stacking')

Transformation Pipeline and Model Successfully Loaded


In [103]:
# predict
result = predict_model(stacker, data=df_test, raw_score=False)   # predict
result['Label'] = result['Label'].astype('float64').astype('int64')

In [104]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result['Label'].values.reshape(-1,1)
sub.to_csv(f'{PROJECT_NAME}_stacking.csv', index=False)

In [105]:
# load blending model
blender = load_model(PROJECT_NAME + '_blending')

Transformation Pipeline and Model Successfully Loaded


In [106]:
# predict
result = predict_model(blender, data=df_test, raw_score=False)   # predict
result['Label'] = result['Label'].astype('float64').astype('int64')

In [107]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result['Label'].values.reshape(-1,1)
sub.to_csv(f'{PROJECT_NAME}_blending.csv', index=False)